# JETANK Gesture Controlling

The document is run to display camera feed and set up a TCP/IP socket that recieves command from a client. The commands can change mode for the movement of the JETANK or steer the movement itself. Run all cells but the last and the JETANK is good to go.

The document is influenced by the gamepadCtrl Notebook. (Source: https://github.com/waveshare/JETANK)

### Import dependencies and displaying camera feed

Running the code below will instanciate the camera and link the feed to a display widget.

In [ ]:
import socket
import threading
import time
from time import sleep
from jetbot import Robot
from SCSCtrl import TTLServo
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

# Creating a connection between the Jetson camera and a display widget
camera = Camera.instance(width=300, height=300)

image = widgets.Image(format='jpeg', width=500, height=500)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

## Functions & Variables
 Below all functions and variables of the program, regarding the movement of the robot are listed 

In [ ]:
robot = Robot()
serverPort = 12000
serverPort2 = 1234

# create TCP socket and bind to specified port
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind(('', serverPort))
s.listen()

print ("TCP SERVER LISTENING")

def move_forward():
    robot.forward(0.9)
    time.sleep(0.5)
    robot.forward(0.7)
    time.sleep(0.5)
    robot.stop()

def stop():
    robot.stop()

def move_backward():
    robot.backward(0.7)
    time.sleep(0.5)
    robot.backward(0.4)
    time.sleep(0.5)
    robot.stop()

def move_left():
    robot.left(0.5)
    time.sleep(0.3)
    robot.stop()


def move_right():
    robot.right(0.5)
    time.sleep(0.3)
    robot.stop()
        
robot_mode = 'move'

servoCtrlTime = 0.001  # Delay after the TTL servo communication to avoid errors.

armXStatus = 0         # The movement state of the X axis of the robotic arm (far and near direction).
armYStatus = 0         # The movement state of the Y axis of the robotic arm (vertical direction).
cameStatus = 0         # The movement state of the Camera.

goalX = 130            # Store the X-axis coordinate of the gripper of the robotic arm.
goalY = 50             # Store the Y-axis coordinate of the gripper of the robotic arm.

xMax = 210             # Set the maximum X axis of the robotic arm.
xMin = 140             # Set the minimum X axis of the robotic arm.

yMax = 120             # Set the maximum Y axis of the robotic arm.
yMix = -120            # Set the minimum Y axis of the robotic arm.

movingTime = 0.005     # Set the running time between every two adjacent positions of the inverse kinematics function of the robotic arm.
movingSpeed = 20        # Set the movement speed of the robotic arm.

grabStatus = 0         # The movement state of the gripper.

# Control the gripping and loosening of the gripper.
def changeMode(mode):
    global robot_mode
    if not controlTank.mode_pressed:
        print("Not Changing Mode")
    else:
        if robot_mode == 'move':
            robot_mode = 'arm'
        elif robot_mode == 'arm':
            robot_mode = 'move'
        controlTank.mode_pressed=0

# Determines what sort of move should be performed based on current mode
def determine_move(command):
    global robot_mode   
    if robot_mode == 'move':
        moveJetank(command)
    elif robot_mode == 'arm':
        if command == 'w' or command == 'a' or command == 's' or command == 'd':
            armCtrl(command)
        elif command == 'g' or command == 'l':
            grabCtrl(command)

# Calls move functions respective to the input
def moveJetank(button_pressed):
    if not controlTank.new_pressed:
        stop()
    else:
        if button_pressed == "w":
            move_forward()
        elif button_pressed == "s":
            move_backward()
        elif button_pressed == "a":
            move_left()
        elif button_pressed == "d":
            move_right()
    controlTank.new_pressed = 0


# Control the gripping and loosening of the gripper.
def grabCtrl(commandType):
    global grabStatus
    if not controlTank.new_pressed:
        print("Not Grabbin'")
    else:
        if commandType == 'g':
            commandLoose = 0
            commandGrab = 1
        elif commandType == 'l':
            commandGrab = 0
            commandLoose = 1

        if commandGrab and not grabStatus:
            TTLServo.servoAngleCtrl(4, -70, 1, 150)
            grabStatus = 1
            sleep(servoCtrlTime)
        elif commandLoose and grabStatus:
            TTLServo.servoAngleCtrl(4, 0, 1, 150)
            grabStatus = 0
            sleep(servoCtrlTime)
            controlTank.new_pressed=0
    
# Robotic arm control function, for action execution (in another single thread).
def armCtrl(button_pressed):
    global xMax, xMin, yMax, yMix, goalX, goalY, armXStatus, armYStatus
    if not controlTank.new_pressed:
        print("Not Moving arm")
    else:
        if button_pressed == "w":
            armXStatus = 1
            goalX += movingSpeed
            if goalX > xMax:
                goalX = xMax
            TTLServo.xyInputSmooth(goalX, goalY, movingTime)
            sleep(servoCtrlTime)
            controlTank.new_pressed = 0
            return
        elif button_pressed == "s":
            armXStatus = 1
            goalX -= movingSpeed
            if goalX < xMin:
                goalX = xMin
            TTLServo.xyInputSmooth(goalX, goalY, movingTime)
            sleep(servoCtrlTime)
            controlTank.new_pressed = 0
            return
        
        elif button_pressed == "d":
            armYStatus = 1
            goalY += movingSpeed
            if goalY > yMax:
                goalY = yMax
            if goalY > -50:
            TTLServo.xyInputSmooth(goalX, goalY, movingTime)
            sleep(servoCtrlTime)
            controlTank.new_pressed = 0
            return
            
        elif button_pressed == "a":
            armYStatus = 1
            goalY -= movingSpeed
            if goalY < yMix:
                goalY = yMix
            if goalY > -50:
            TTLServo.xyInputSmooth(goalX, goalY, movingTime)
            sleep(servoCtrlTime)
            controlTank.new_pressed = 0
            return


TTLServo.xyInputSmooth(goalX, goalY, 3)
TTLServo.servoAngleCtrl(1, 0, 1, 100)
TTLServo.servoAngleCtrl(4, 0, 1, 100)
TTLServo.servoAngleCtrl(5, 0, 1, 100)
sleep(3.5)


## Control Thread
Create controlThread responsible for the movement of the robot.

In [3]:
class controlThread(threading.Thread):
    def __init__(self, *args, **kwargs):
        super(controlThread, self).__init__(*args, **kwargs)
        self.__flag = threading.Event()
        self.__flag.clear()
        self.button_pressed = ""
        self.new_pressed = 0
        self.mode_pressed = 0

    def pause(self):
        self.__flag.clear()

    def resume(self):
        self.__flag.set()

    def run(self):
        while 1:
            self.__flag.wait()

            # Calls functions based on variable set in Input thread
            if self.new_pressed:
                determine_move(self.button_pressed)
            elif self.mode_pressed:
                changeMode(self.button_pressed)
            if not self.new_pressed and not self.mode_pressed:
                self.pause()
            sleep(movingTime)


# Instantiate and start the multi-threaded servo control thread.
controlTank = controlThread()
controlTank.start()

## Input Thread
Create inputThread responsible for the socket connection and input received

In [ ]:
class inputThread(threading.Thread):
    def __init__(self, *args, **kwargs):
        super(inputThread, self).__init__(*args, **kwargs)
        self.__flag = threading.Event()
        self.__flag.clear()

    def pause(self):
        self.__flag.clear()

    def resume(self):
        self.__flag.set()

    def run(self):
        while 1:
            conn, addr = s.accept()
            with conn:
                print("Connected to: ")
                print(addr)
                while 1:
                    
                    self.__flag.wait()
                    message, clientAddress = conn.recvfrom(2048)
                    mess = message.decode()

                    # Look for specifik input to steer the Jetank accordingly
                    if mess == "w" or mess == "a" or mess == "s" or mess == "d" or mess == "g" or mess == "l":
                        print("Changing move variables")
                        controlTank.button_pressed = mess
                        controlTank.new_pressed = 1
                        controlTank.resume()
                    
                    # Changes mode for the movement of the robot
                    elif mess == "c" or mess == "x":
                        print("Changing mode variables")
                        controlTank.button_pressed = mess
                        controlTank.mode_pressed = 1
                        controlTank.resume()


# Instantiate the gamepad input thread and start reading information
inputThreading = inputThread()
inputThreading.start()
inputThreading.resume()


In [6]:
camera.stop()